In [1]:
import pyexistdb.db

In [7]:
test = pyexistdb.db.ExistDB(server_url = 'http://localhost:8080/exist/xmlrpc')

In [8]:
print(test)

In [14]:
test.getCollectionDescription('apps/demo/data')

{'owner': 'demo',
 'collections': ['addresses', 'binary', 'i18n'],
 'documents': [{'name': 'r_and_j.xml',
   'owner': 'demo',
   'type': 'XMLResource',
   'permissions': 509,
   'group': 'demo'},
  {'name': 'mondial.xml',
   'owner': 'demo',
   'type': 'XMLResource',
   'permissions': 509,
   'group': 'demo'},
  {'name': 'macbeth.xml',
   'owner': 'demo',
   'type': 'XMLResource',
   'permissions': 509,
   'group': 'demo'},
  {'name': 'hamlet.xml',
   'owner': 'demo',
   'type': 'XMLResource',
   'permissions': 509,
   'group': 'demo'},
  {'name': 'shakes.xsl',
   'owner': 'demo',
   'type': 'XMLResource',
   'permissions': 509,
   'group': 'demo'}],
 'created': '1576494179903',
 'permissions': 509,
 'name': '/db/apps/demo/data',
 'group': 'demo'}

['DEFAULT_TIMEOUT',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_collectionIndexPath',
 '_configCollectionName',
 '_init_from_xmlrpc_url',
 '_serverurl_from_djangoconf',
 'createCollection',
 'create_account',
 'create_group',
 'describeDocument',
 'encoding',
 'executeQuery',
 'exist_url',
 'getCollectionDescription',
 'getDoc',
 'getDocument',
 'getHits',
 'getPermissions',
 'hasCollection',
 'hasCollectionIndex',
 'hasDocument',
 'load',
 'loadCollectionIndex',
 'moveDocument',
 'password',
 'query',
 'querySummary',
 'reindexCollection',
 'releaseQueryResult',
 'removeCollection',
 'removeCollectionIndex',
 'removeDocument',
 'restapi_path',
 'resultType',
 'r